In [ ]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)


## Lecture 12 ##

## A Join Example ##

In [ ]:
full = Table.read_table('nc-est2019-agesex-res.csv')
census = full.select('SEX', 'AGE', 'POPESTIMATE2019')
census.show(3)

In [ ]:
sex_codes = Table().with_columns(
    'SEX CODE', make_array(0, 1, 2),
    'CODE DEFINITION', make_array('All', 'Selected Male', 'Selected Female')
)
sex_codes

In [ ]:
sex_codes.join('SEX CODE', census, 'SEX').sort('AGE').show(3)

## Bike Sharing ##
Describe the incoming data set

Note: Duration is in seconds. 

In [ ]:
trip = Table.read_table('trip.csv')
trip.show(15)

## Distribution of Durations ##

In [ ]:
trip.hist('Duration')

Review the horizontal axis. Any guesses as to why the values are so high?

Let's check the data...

In [ ]:
trip.sort('Duration', descending=True)

What could have caused those high duration values? (Remember, duration is in seconds.)


Let's limit the duration to a realistic value. 
Check the limit below, what is that in minutes?

In [ ]:
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')

Describe the shape of the histogram.

What does that tell us about the data?

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

The histogram above is the same data as the previous histogram but displayed differently.
What was changed and how was it changed?

Which view do you prefer and why?

In [ ]:
# Approx percent of people who have # a ride duration between 250 and 500 seconds
# "between" = [250, 500) 
# Recall: The area of the histogram bar gives on the percent of people. 
# (since the vertical axis is the percent per second)

(500-250) * 0.15 

In [ ]:
# Determine how many commutes were in the 250 to 500 category.

count_250 = commute.where('Duration', are.between(250, 500)).num_rows
count_250

In [ ]:
# Determine the total number of commutes recorded in this sample.

total_commutes = commute.num_rows
total_commutes

In [ ]:
# Determine the decimal value of that ratio. 
# How close is it to the histograms display?
count_250 / total_commutes

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
commute.hist('Duration', bins=60, unit='Second')

Describe the new histogram.

What changed?

Which display of the three would you prefer to view to interpret this data set? Why?

## Start and End Stations ##

In [ ]:
# Most common start station

starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
# Numbers of trips between stations

commute.pivot('Start Station', 'End Station')

How do you read the above table?



In [ ]:
# Average durations of trips between stations

commute.pivot('Start Station', 'End Station', values='Duration', collect=np.average)

Decribe the difference between the two tables above.

What observations can be made from the different tables. 


## Fastest Trips between Stations ##

How can we find the fastest trip ever between each pair of stations?

Start by creating a new table to display the information we need to answer this question.

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
shortest = duration.group(['Start Station', 'End Station'], min)
shortest.show(5)

What do you notice about the fastest (or shortest) duration stations?)

## Discussion question

Find the 5 stations closest to Civic Center BART by minimum trip time.

In [ ]:
# Complete the code
from_cc = 

from_cc.take(np.arange(5))

## Maps ##

We can also create map displays if we have enough location data.

In [ ]:
# Geographical data on the stations
stations = Table.read_table('station.csv').drop(4, 6)
stations

In [ ]:
# Create a new table that does what?

sf_stations = stations.where('landmark', are.equal_to('San Francisco'))
sf_stations_map_data = (sf_stations
 .select('lat', 'long', 'name')
 .relabeled('name', 'labels'))
sf_stations_map_data.show(3)

In [ ]:
# Display the data as markers on a map.

Marker.map_table(sf_stations_map_data)

In [ ]:
# Display the data as circles on the map. 
Circle.map_table(sf_stations_map_data)

### Discussion question

Map all stations within 4 minutes (minimum ride time) of Civic Center.

In [ ]:
from_cc.show(3)

In [ ]:
sf_stations_map_data.show(3)

In [ ]:
#Complete the code
close_cc = from_cc.where('Duration min', are.below(4 * 60))
close_markers = (
    sf_stations_map_data
        .join('labels',close_cc, 'End Station')
        .select('lat','long','labels'))
#close_markers

Marker.map_table(close_markers)

What does the map above display?


Choose marker colors by the minimum time from Civic Center

In [ ]:
minutes = np.round(from_cc.column("Duration min") / 60)
print(min(minutes), max(minutes))

In [ ]:
colors = Table().with_columns(
    "minutes", np.arange(15),
    "colors",  ["darkblue", "blue", "lightblue", 
                "darkgreen", "green", "lightgreen",
                "orange", "darkred", "red",
                "gray", "gray", "gray", 
                "gray", "gray", "gray"])
colors_cc = (from_cc
 .with_column("Minutes", minutes)
 .join("Minutes", colors, "minutes"))

colored_markers = (sf_stations_map_data
      .join('labels', colors_cc, 'End Station')
      .select('lat', 'long', 'labels', 'colors'))
Marker.map_table(colored_markers)

There is a lot of code in the last block. What is happening in the code?

What is the output final display showing?

Use join to create a new table below. What is being joined?

What does the "count" column represent?


In [ ]:
station_starts = stations.join('name', starts, 'Start Station')
station_starts.show(3)

In [ ]:
sf_starts = station_starts.relabeled(
    'name', 'labels').where('landmark', 'San Francisco')
sf_starts

In [ ]:
sf_starts_map_data = sf_starts.select('lat', 'long', 'labels').with_columns(
    'colors', 'blue',
    'areas', sf_starts.column('count') / 10
)
sf_starts_map_data.show(5)

In [ ]:
Circle.map_table(sf_starts_map_data)

Why are the circles different sizes?

Can you find the smallest circle? Display the table data differently to identify which station that is. 
Why does that location have the smallest circle?
